# Practical 3: Deep Feed-forward Networks
**IndabaX sudan 2019**

## Introduction
In this practical, we implement and train a feed-forward neural network (also known as an "MLP" for "multi-layer perceptron") on a dataset called **"Fashion MNIST"**, consisting of small greyscale images of items of fashion. We consider the practical issues around generalisation to out-of-sample data and introduce some important techniques for addressing this.

Before diving into the Neural network part we first have an introduction to Tensorflow, a very popular deep learning framework that is used in all aspects of defining, training and testing models. 
## Learning objectives
* Understand the basics of Tensorflow
* Use **Tensorflow Eager** and **Keras Layers** to build a neural network archetecture
* Understand how a model is trained and evaluated
* Differentiate between **in-sample** and **out-of-sample** model performance
* Understand the concept of **train/validation/test split** and why it's useful
* Research at least 1 techniques that can be used to improve model **generalisation**

# Tensorflow
As described in Wikipedia TensorFlow is a computational framework for building machine learning models. It is the second generation system from Google Brain headed by Jeff Dean. Launched in early 2017, it has disrupted the ML world by bringing in numerous capabilities from scalability to building production ready models.

In other words, Tensorflow is a python library that provides matrix and list functionality and can work on multiple level of abstraction across multiple languages. Tensorflow is available on C++, python and recently on JavaScript.

In [1]:
#@title Imports (RUN ME!) { display-mode: "form" }

!pip install tensorflow-gpu==2.0.0-beta0 > /dev/null 2>&1

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

print("TensorFlow executing eagerly: {}".format(tf.executing_eagerly()))

The system cannot find the path specified.


ModuleNotFoundError: No module named 'tensorflow'